In [21]:
using CSV, Tables
using JuMP
using Gurobi

import Pkg; Pkg.add("JSON3")
using JSON3

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [7]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-08


## load in data

In [ ]:
all_classes = open("data/all_classes_info.json", "r") do file 
    JSON3.read(file)
end

R = CSV.File("data/ratings.csv",header=0) |> Tables.matrix;
H = CSV.File("data/hours.csv",header=0) |> Tables.matrix;
U = CSV.File("data/units.csv",header=0) |> Tables.matrix;

vars = CSV.File("data/variables.csv",header=0) |> Tables.matrix;
n = vars[1];
S = vars[2];

In [24]:
all_classes[1]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 28 entries:
  :rating              => 5.4
  :gir_attribute       => "REST"
  :has_final           => false
  :description         => "Presents engineering problems in a computational set…
  :offered_fall        => false
  :offered_spring      => true
  :meets_with_subjects => ["1.001"]
  :instructors         => ["J. Williams"]
  :out_of_class_hours  => 7.99
  :total_units         => 12
  :related_subjects    => ["2.156", "1.205", "1.C51", "1.000", "1.631", "1.063"…
  :pdf_option          => false
  :in_class_hours      => 5.7
  :is_half_class       => false
  :level               => "U"
  :prerequisites       => "GIR:CAL1"
  :subject_id          => "1.00"
  :title               => "Engineering Computation and Data Science"
  :lab_units           => 2
  :design_units        => 0
  :public              => true
  :offered_summer      => false
  :lecture_units       => 3
  :

In [19]:
all_classes[1]

"{\n                \"rating\": 5.4,\n         \"gir_attribute\": \"REST\",\n             \"has_final\": false,\n           \"description\": \"Presents engineering problems in a computational setting with emphasis on data science and problem abstraction. Covers exploratory data analysi" ⋯ 1448 bytes ⋯ "true,\n        \"offered_summer\": false,\n         \"lecture_units\": 3,\n     \"preparation_units\": 7,\n     \"enrollment_number\": 25.43,\n                   \"url\": \"http://student.mit.edu/catalog/m1a.html#1.00\",\n     \"is_variable_units\": false,\n           \"offered_IAP\": false\n}"

In [13]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV))

α = 0.8

# @variable(model, X[1:n,1:s] >=0)
@variable(model, X[1:n, 1:S], Bin)

@objective(model, Min, sum(α*H[i]*X[i,s] - (1-α)*R[i]*X[i,s] for i=1:n,s=1:S));

# full-time; at least 36 units per semester
@constraint(model, [s in 1:S], sum(U[i]*X[i, s] for i=1:n) >= 36);
# wellbeing: no more than 112 hours
@constraint(model, [s in 1:S], sum(H[i]*X[i, s] for i=1:n) <= 112);
# not repeatable: cannot take a class again
# TODO

optimize!(model)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 37624 columns and 70984 nonzeros
Model fingerprint: 0x22f7e491
Variable types: 0 continuous, 37624 integer (37624 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [2e-02, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+02]
Found heuristic solution: objective 153.5920000
Presolve removed 14 rows and 34443 columns
Presolve time: 0.17s
Presolved: 2 rows, 3181 columns, 6360 nonzeros
Found heuristic solution: objective 17.8000000
Variable types: 0 continuous, 3181 integer (2476 binary)
Found heuristic solution: objective 14.6400000

Root relaxation: objective 4.125000e+00, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 E

In [14]:
taken_classes_sem = [[ind for (ind, val) in enumerate(sem) if val == 1] for sem in eachcol(value.(X))]

8-element Vector{Vector{Int64}}:
 [294, 714, 816, 1180, 2021, 2885, 3017, 4347]
 [294, 714, 816, 1180, 2021, 2885, 3017, 4347]
 [294, 714, 816, 1180, 2021, 2885, 3017, 4347]
 [144, 294, 816, 1180, 2021, 2885, 3017, 4347]
 [144, 294, 816, 1180, 2021, 2885, 3017, 4347]
 [294, 714, 816, 1180, 2021, 2885, 3017, 4347]
 [144, 294, 816, 1180, 2021, 2885, 3017, 4347]
 [294, 714, 816, 1180, 2021, 2885, 3017, 4347]

## Interpreter

In [15]:
function interpret_course_nums(classes_id_per_sem)
    course_nums = [[all_classes[class_ind][:subject_id] for class_ind in sem] for sem in classes_id_per_sem]
    return course_nums
end

interpret_course_nums (generic function with 1 method)

In [26]:
interpret_course_nums(taken_classes_sem)

8-element Vector{Vector{String}}:
 ["10.960", "12.834", "14.399", "15.839", "20.S900", "24.93", "3.903", "HST.533"]
 ["10.960", "12.834", "14.399", "15.839", "20.S900", "24.93", "3.903", "HST.533"]
 ["10.960", "12.834", "14.399", "15.839", "20.S900", "24.93", "3.903", "HST.533"]
 ["1.713", "10.960", "14.399", "15.839", "20.S900", "24.93", "3.903", "HST.533"]
 ["1.713", "10.960", "14.399", "15.839", "20.S900", "24.93", "3.903", "HST.533"]
 ["10.960", "12.834", "14.399", "15.839", "20.S900", "24.93", "3.903", "HST.533"]
 ["1.713", "10.960", "14.399", "15.839", "20.S900", "24.93", "3.903", "HST.533"]
 ["10.960", "12.834", "14.399", "15.839", "20.S900", "24.93", "3.903", "HST.533"]